In [ ]:
#  导入需要的库
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # 导入time模块


# from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException 
from selenium.common.exceptions import StaleElementReferenceException , ElementClickInterceptedException

import pandas as pd

import csv
from selenium.webdriver.support.ui import Select
from selenium.webdriver.edge.options import Options # 导入 Edge 浏览器选项模块


In [ ]:
# 登录模块
def login_control():
    driver = webdriver.Chrome()
    wait_3000 = WebDriverWait(driver, 3000)

    # 登录
    driver.get("https://analyst.lightcast.io/analyst/?t=4D2MH#page=dashboard&login=1") 
    # 等待页面加载完成
    wait_30 = WebDriverWait(driver, 300) 
    # 获取用户名输入框，密码输入框和log in按钮的元素
    username_box = wait_3000.until(EC.presence_of_element_located((By.ID, "userinput"))) # 等待用户名输入框出现，并获取它的元素，这里替换成用户名输入框的id属性值
    password_box = wait_3000.until(EC.presence_of_element_located((By.ID, "passwordinput"))) # 等待密码输入框出现，并获取它的元素，这里替换成密码输入框的id属性值
    login_button = wait_3000.until(EC.element_to_be_clickable((By.ID, "submitbutton"))) # 等待log in按钮可点击，并获取它的元素，这里替换成log in按钮的id属性值
    # 输入你的用户名和密码，并点击log in按钮
    username_box.send_keys("username") # 输入你的用户名
    password_box.send_keys("password") # 输入你的密码
    # 使用JavaScript代码来点击log in
    driver.execute_script("arguments[0].click();", login_button)

    # 点击按钮report
    report_button = wait_3000.until(EC.element_to_be_clickable((By.ID, "report-dropdown"))) # 等待group按钮可点击，并获取它的元素
    report_button.click() 
    
    # 在report弹出的导航列表里选择Job Postings，并跳转到该页面
    jobpositings_link = wait_3000.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/ul[1]/li[2]/div/ul/li[1]/a"))) 

    jobpositings_link.click() 
    postings_table_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div[1]/div/a[2]")))
    postings_table_button.click()
    # run
    run_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div/button")))
    driver.execute_script("arguments[0].click();", run_button)
    return driver


# 选择公司的组别
def chose_company_group(i,driver):
    wait_3000 = WebDriverWait(driver, 3000)

    company_group_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div[1]/div/div[5]/h3/a[contains(text(),"Groups")]'))) 
    driver.execute_script("arguments[0].click();", company_group_button)

    groupselesct_xpath = "/html/body/div[5]/div[2]/div/div[5]/div/div[2]/div/div[1]/div/span/div[2]/a[" + str(i) + "]"
    groupselect_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH, groupselesct_xpath)))
    driver.execute_script("arguments[0].click();", groupselect_button)    

    select_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/div/div[5]/div/div[2]/div/div[2]/button[1][contains(text(),'Select')]")))
    driver.execute_script("arguments[0].click();", select_button)
    return driver

# 程序中之后需要修改
# 用于出错后，文件中止，跳过前面已经下载过的筛选条件的文件
def logic(index):
    # index 是从0开始的，需要跳过x行，则index<x
 if index < 0 : # 跳过前97行,从第98行开始
  return True
 return False


# 开始进行筛选条件的选择与导出 
# select_.csv 文件是所有的筛选情况，这样做可以降低程序的内存，根据读csv文件来控制选择
def selection(row,flag,driver,study_level,job_type,pf_time,job_location,Posting_type,Minimum_experience):
    wait_3000 = WebDriverWait(driver, 3000) 
    wait_5 = WebDriverWait(driver, 10) 
    wait_min_5 = WebDriverWait(driver, 300)
    # 筛选条件  Minimum experience      
    Minimum_Experience_required_xapth_any = '/html/body/div[5]/div[1]/div/div[14]/div/div/input[1]'
    Minimum_Experience_required_xapth_select_range = '/html/body/div[5]/div[1]/div/div[14]/div/div/input[2]'
    any_buttons =wait_3000.until(lambda driver: driver.find_element(By.XPATH, Minimum_Experience_required_xapth_any))
    range_buttons =wait_3000.until(lambda driver: driver.find_element(By.XPATH, Minimum_Experience_required_xapth_select_range))
    # 筛选条件 最低工作经验
    if Minimum_experience == '0':
        driver.execute_script("arguments[0].click();", any_buttons)
        Minimum_experience_print = "minimum experience required: any"
    else:
        driver.execute_script("arguments[0].click();", range_buttons)
        Minimum_experience_print = "minimum experience required: select ranges"
    time.sleep(3)
        
    # 筛选条件 学历
    if study_level != '6':
        checkbox = wait_3000.until(EC.presence_of_element_located((By.ID, f"edulevels_{study_level}")))
        driver.execute_script("arguments[0].click();", checkbox)
        study_level_print = f"education levels chose {study_level}"
    else:   
        for study_level in range(6):
            checkbox = wait_3000.until(EC.presence_of_element_located((By.ID, f"edulevels_{study_level}")))
            driver.execute_script("arguments[0].click();", checkbox)
            study_level_print = 'all education level'
    
    # 筛选条件 Job Tpye           
    select_job_type= Select(wait_3000.until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[5]/div[1]/div/div[17]/label/select"))))        
        # 动作区
    if job_type == '0':
        select_job_type.select_by_visible_text("Include Internships")
        job_type_print="job type:Include Internships"
    else:
        select_job_type.select_by_visible_text("Exclude Internships")
        job_type_print="job type:Exclude Internships"
        
    # 筛选条件 part-time / full-time
    time.sleep(3)
    FP_TIME_checkbox = wait_3000.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="option_form"]/div[18]/ul/li['+str(pf_time)+']')))      
    driver.execute_script("arguments[0].click();", FP_TIME_checkbox)
    PF_TIME_print =  f"part-time / full-time chose {pf_time}"
    # 筛选条件 job location :
    if job_location == '3':
        for i in range(1,5):
            job_location_checkbox = wait_3000.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="option_form"]/div[19]/ul/li['+str(i)+']')))      
            driver.execute_script("arguments[0].click();", job_location_checkbox)
            job_location_print= "job location:all"
    else:
        job_location_checkbox = wait_3000.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="option_form"]/div[19]/ul/li['+str(job_location)+']')))      
        driver.execute_script("arguments[0].click();", job_location_checkbox)
        job_location_print= f"job location chose {job_location}"
        
    # 筛选条件 posting type
    posting_type_select= Select(wait_3000.until(EC.element_to_be_clickable((By.ID,"postings_type_input"))))
    # 动作区
    if Posting_type =='0':
        posting_type_select.select_by_visible_text("Active Postings")
        Posting_type_print="posting type:Active Postings"
    else:
        posting_type_select.select_by_visible_text("Newly Posted")
        Posting_type_print="posting type:Newly Posted"
    company_group_clear_button = wait_5.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div[1]/div/div[5]/h3/a[contains(text(),"Clear")]')))
    
    # 导出模块
    output_selectall_button_xpath = "/html/body/div[5]/div[3]/div[3]/div[5]/div/div/table/thead/tr/th[1]"
    output_selectall_button = wait_3000.until(EC.element_to_be_clickable((By.XPATH, output_selectall_button_xpath)))
    
    driver.execute_script("arguments[0].click();", output_selectall_button)
    out_csv_xpath1= "/html/body/div[5]/div[3]/div[1]/div/a"
    out_csv_excel_xpath = '/html/body/div[5]/div[3]/div[1]/div/div/ul/li[2]/a'
    out_csv_xpath2="/html/body/div[5]/div[3]/div[1]/div/div/ul/li[3]/a"
    
    time.sleep(5)
    # 导出csv
    out_csv_button1 = wait_3000.until(EC.element_to_be_clickable((By.XPATH, out_csv_xpath1)))
    driver.execute_script("arguments[0].click();", out_csv_button1)
    out_csv_button2 = wait_3000.until(EC.element_to_be_clickable((By.XPATH, out_csv_xpath2)))
    driver.execute_script("arguments[0].click();", out_csv_button2)
    
    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle!=main_window:
            driver.switch_to.window(handle)
    try : 
        direct = wait_min_5.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[@class="direct"]')))
    except:
            print('导不出来csv文件[记得这个flag需要加1才是导不出来的那个文件]flag = ', flag)   
            # 记录导不出来的文件，后面再手动导出
    finally:
        driver.switch_to.window(main_window)
    
    time.sleep(2)
    # 导出excel
    out_csv_button1 = wait_3000.until(EC.element_to_be_clickable((By.XPATH, out_csv_xpath1)))
    driver.execute_script("arguments[0].click();", out_csv_button1)
    out_csv_excel_botton = wait_3000.until(EC.element_to_be_clickable((By.XPATH, out_csv_excel_xpath)))
    driver.execute_script("arguments[0].click();", out_csv_excel_botton)
    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle!=main_window:
            driver.switch_to.window(handle)
    try : 
        direct = wait_min_5.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[@class="direct"]')))
    except:
            print('导不出来excel文件[记得这个flag需要加1才是导不出来的那个文件]flag = ', flag)   
            # 记录导不出来的文件，后面再手动导出
    finally:
        driver.switch_to.window(main_window)     
    flag = flag+1
  
    # 更换公司后需要修改
    with open('Selection_group_5.csv', 'a', newline='') as f:
        # 创建一个csv写入器
        writer = csv.writer(f)

        row.extend([flag,study_level_print,job_type_print,PF_TIME_print,job_location_print,Posting_type_print,Minimum_experience_print])
        # 将列表作为一行写入csv文件
        writer.writerow(row)
        f.close()
        
    time.sleep(2)
    
    # job location clear
    clear_job_location = wait_min_5.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="remote_clear"]')))      
    driver.execute_script("arguments[0].click();", clear_job_location)
    time.sleep(2)

    # 筛选条件3 part-time / full-time clear
    claer_FP_TIME_checkbox = wait_min_5.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="employment_type_clear"]'))) 
    driver.execute_script("arguments[0].click();", claer_FP_TIME_checkbox)
    time.sleep(2)

    # 清除Education_Levels选项，为下次选择做准备   
    Education_Levels_xapth_claer = '/html/body/div[5]/div[1]/div/div[16]/h3/a'
    Education_Levels_claer_button = wait_min_5.until(EC.element_to_be_clickable((By.XPATH, Education_Levels_xapth_claer)))
    driver.execute_script("arguments[0].click();", Education_Levels_claer_button)
    error = True
    
    return error, flag

def loopfordown():
    # 这是已经登录好了，未选择公司的浏览器
    driver = login_control()
    
    # 更换公司后需要修改
    # 选择公司，此处是选择第6个公司，即 group_5 (group从0开始)
    driver = chose_company_group(6,driver)
    with open('select_.csv','r') as selectfile:
        
        Order = csv.reader(selectfile)
        # 出了错，假设是需要从第二列序号为【3】的文件开始，序号为3的文件已经下载过，不需要下载的情况下，flag = 【总共下载的文件数量，查看生成的'Selection_group_5.csv'文件】 logic(index) 函数需要控制读取select_.csv跳过前3行，如果需要重新下载，则logic(index) 函数需要控制读取select_.csv跳过前2行。
        # flag 用于记录已经下载了几个文件
        flag = 0
        for index, row in enumerate(Order):
            if not logic(index):
                error = False
                while not error: 
                    # 当标志变量为False时，执行以下代码
                    study_level = row[0]
                    job_type = row[1]
                    pf_time = row[2]
                    job_location = row[3]
                    Posting_type = row[4]
                    Minimum_experience = row[5]
                    try:
                        error,flag = selection(row,flag,driver,study_level,job_type,pf_time,job_location,Posting_type,Minimum_experience)
 
                    except: 
                        # 如果出现错误，执行以下代码
                        print("在flag",flag,"处出现错误")
                        # 退出浏览器并重新登录和选择公司
                        driver.quit()   
                        driver = login_control()
                        
                        # 更换公司后需要修改
                        driver = chose_company_group(6,driver)
                        # 不改变标志变量，不退出while循环
        

# 主要运行的函数：
# 开始进行循环操作
loopfordown()
